In [ ]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # 배치를 제외한 모든 차원을 평탄화(flatten)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [ ]:
def test(model, device, test_loader, model_name):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('{} Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        model_name, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    model.train()

In [ ]:
def train(args, model, mean_teacher, device, train_loader, test_loader, optimizer, epoch):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        output = model(data)

        with torch.no_grad():
            mean_t_output = mean_teacher(data)

        const_loss = F.mse_loss(output, mean_t_output)

        weight = 0.2
        loss = F.nll_loss(output, target) + weight*const_loss
        loss.backward()
        optimizer.step()

        alpha = 0.95
        for mean_param, param in zip(mean_teacher.parameters(), model.parameters()):
            mean_param.data.mul_(alpha).add_(1 - alpha, param.data)

        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))
            test(model, device, test_loader, "STUDENT")
            test(mean_teacher, device, test_loader, "MEAN_TEACHER")
            print()


In [ ]:
def main():
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')
    parser.add_argument('--epochs', type=int, default=10, metavar='N',
                        help='number of epochs to train (default: 10)')
    parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                        help='learning rate (default: 0.01)')
    parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                        help='SGD momentum (default: 0.5)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=100, metavar='N',
                        help='how many batches to wait before logging training status')

    parser.add_argument('--save-model', action='store_true', default=False,
                        help='For Saving the current Model')
    parser.add_argument('-f')

    args = parser.parse_args()
    use_cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('../data', train=True, download=True,
                       transform=transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(p=0.3),
        transforms.RandomVerticalFlip(p=0.3),
        transforms.GaussianBlur(3),
        transforms.RandomErasing(p=0.3),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)),
        batch_size=args.batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('../data', train=False, transform=transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(p=0.3),
        transforms.RandomVerticalFlip(p=0.3),
        transforms.GaussianBlur(3),
        transforms.RandomErasing(p=0.3),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)),
        batch_size=args.test_batch_size, shuffle=True, **kwargs)

    model = Net().to(device)

    mean_teacher = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

    for epoch in range(1, args.epochs + 1):
        train(args, model, mean_teacher, device, train_loader, test_loader, optimizer, epoch)

    if (args.save_model):
        torch.save(model.state_dict(), "cifar10_cnn.pt")

In [ ]:
if __name__ == '__main__':
    main()

Files already downloaded and verified
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.299417
STUDENT Test set: Average loss: 2.3038, Accuracy: 965/10000 (10%)
MEAN_TEACHER Test set: Average loss: 2.3047, Accuracy: 990/10000 (10%)

Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.302548
STUDENT Test set: Average loss: 2.3026, Accuracy: 1012/10000 (10%)
MEAN_TEACHER Test set: Average loss: 2.3028, Accuracy: 1077/10000 (11%)

Train Epoch: 1 [12800/50000 (26%)]	Loss: 2.305959
STUDENT Test set: Average loss: 2.3013, Accuracy: 997/10000 (10%)
MEAN_TEACHER Test set: Average loss: 2.3015, Accuracy: 1009/10000 (10%)

Train Epoch: 1 [19200/50000 (38%)]	Loss: 2.294536
STUDENT Test set: Average loss: 2.2996, Accuracy: 1080/10000 (11%)
MEAN_TEACHER Test set: Average loss: 2.3000, Accuracy: 1022/10000 (10%)

Train Epoch: 1 [25600/50000 (51%)]	Loss: 2.302563
STUDENT Test set: Average loss: 2.2962, Accuracy: 1443/10000 (14%)
MEAN_TEACHER Test set: Average loss: 2.2968, Accuracy: 1398/10000 (14%)

Train Epoch: 1 [32000